In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

 the standard Newton's method formula for finding the root of a function \( f(x) = 0 \) is:

$$
x_{n} = x_{n-1} - \frac{f(x_{n-1})}{f'(x_{n-1})}
$$

This formula is used to iteratively update the value of \( x \) until \( f(x) \) is close to zero. 

However, in the context of optimization, Newton's method is used to find the minimum (or maximum) of a function, not just its root. In this case, the update formula for the parameter vector \( \theta \) is modified accordingly to move towards the minimum of the loss function, hence the formula provided earlier:

$$
\theta_{\text{new}} = \theta_{\text{old}} - \left( \nabla^2 \text{Loss}(\theta_{\text{old}}) \right)^{-1} \nabla \text{Loss}(\theta_{\text{old}})
$$

Here, \( \nabla \text{Loss}(\theta) \) represents the gradient of the loss function, and \( \nabla^2 \text{Loss}(\theta) \) represents the Hessian matrix of the loss function. This modification allows Newton's method to iteratively update the parameters towards the minimum of the loss function in the parameter space.
ameter space.
ameter space.
ameter space.
ameter space.
rameter space.

In the context of logistic regression and Newton's method, let's break down the formulas for calculating the gradient and Hessian matrix:

Gradient of the loss function with respect to the parameters (\( \theta \)):

$$
\nabla Loss(\theta) = \frac{1}{N} X^{T} \cdot (\hat{y} - y)
$$

Where:

- \( X \) is the design matrix (including the bias term).
- \( \hat{y} \) is the predicted probability vector.
- \( y \) is the true label vector.
- \( N \) is the number of samples.

Hessian matrix of the loss function with respect to the parameters (\( \theta \)):

$$
\nabla^{2} Loss(\theta) = \frac{1}{N} X^{T} \cdot D \cdot X
$$

Where:

- \( D \) is a diagonal matrix with elements \( \hat{y}_i \cdot (1 - \hat{y}_i) \) for each sample \( i \).

These formulas represent the first and second derivatives of the loss function with respect to the parameters, which are used in Newton's method for optimization.
method for optimization.


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.special import expit

warnings.filterwarnings("ignore")

# Helper Functions
class Functions:
    ## function for random sampling of data
    def random_sampler(self, perc=0.1):
        prop = int(len(self.X) * perc) #proportion of the data to be used for each iteration, here we have set it to 10% of the data
        ind = np.random.choice(range(len(self.X)), prop, replace=False)
        test_ind = list(set(range(len(self.X))) - set(ind)) #for using the rest sample as test set
        return self.X[ind], self.y[ind], self.X[test_ind], self.y[test_ind]
    
    ## function for normalizing the features
    def normalize_features(self, X, append=True):
        X = (X - np.mean(X, 0)) / np.std(X, 0) #normalize the features
        if append:
            X = np.append(np.ones(X.shape[0]).reshape(-1,1), X, 1) #append column of ones for intercept
        return X


In [13]:
import numpy as np

class LogisticRegression():
    def __init__(self, X, y):
        self.X = X
        self.y = y.values.reshape(-1,1)
        
    def fit(self, max_iter=100, tolerance=1e-6, lr=0.001):         
        # Initialize Beta with small random values
        np.random.seed(0)
        B = np.random.randn(self.X.shape[1], 1) * 0.01
        
        # function for updating parameters using newton's method
        loss_list = []
        test_loss_list = []
        x, y, x_test, y_test = self.random_sampler(perc=0.80)  # use 80-20 in the training too
        
        for it in range(max_iter):
            # calculate training loss
            curr_loss, gradient, hessian = self.cross_entropy_loss(x, y, B)
            loss_list.append(curr_loss)
            
            # calculate test loss
            curr_loss_test, _, _ = self.cross_entropy_loss(x_test, y_test, B)
            test_loss_list.append(curr_loss_test)
            
            # newton method
            e = -np.linalg.solve(hessian, gradient)
            B = B + lr * e
            
            # stop loop once updation becomes insignificant
            if np.all(np.abs(e) < tolerance):
                break
                
        self.B = B
        return B, loss_list, test_loss_list
    
    def predict(self, X):
        scores = np.matmul(X, self.B)
        # Avoiding division by zero
        exp_scores = np.exp(-scores)
        return np.where(exp_scores < 1e-10, 1.0, 1. / (1. + exp_scores))

    def cross_entropy_loss(self, X, y, B):
        N = len(X)
        p = 1. / (1. + np.exp(-np.dot(X, B)))
        W = np.diag((p * (1 - p)).reshape(-1))
        return (-1 / N * (np.dot(y.T, np.log(p)) + np.dot(np.transpose(1 - y), np.log(1 - p))))[0][0], -np.dot(X.T, (y - p)), X.T @ W @ X

    def random_sampler(self, perc=0.8):
        # Randomly shuffle the data
        indices = np.random.permutation(self.X.shape[0])
        split_index = int(self.X.shape[0] * perc)
        train_indices, test_indices = indices[:split_index], indices[split_index:]

        # Split the data into training and testing sets
        x_train, x_test = self.X[train_indices], self.X[test_indices]
        y_train, y_test = self.y[train_indices], self.y[test_indices]
        
        return x_train, y_train, x_test, y_test


In [14]:
## reading the data
log_reg_data = pd.read_csv('logistic.csv')
log_reg_data.head(3)

,Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30
0,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


In [15]:
#Let's replace the categorical variable with M being 1 and B being 0
log_reg_data['Y'] = log_reg_data['Y'].replace({'M':1,'B':0})
train_df, test_df = train_test_split(log_reg_data, test_size=0.2, random_state=42)
train_df.describe()

,Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30
count,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,...,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000
mean,0.371429,14.117635,19.185033,91.882242,654.377582,0.095744,0.103619,0.088898,0.048280,0.181099,...,16.235103,25.535692,107.103121,876.987033,0.131532,0.252742,0.274595,0.114182,0.290502,0.083868
std,0.483719,3.535815,4.266005,24.322027,354.943187,0.013923,0.052470,0.079468,0.038060,0.027487,...,4.811267,6.065108,33.374664,567.672841,0.023083,0.155014,0.209398,0.065326,0.063151,0.017848
min,0.000000,7.691000,9.710000,47.920000,170.400000,0.052630,0.019380,0.000000,0.000000,0.116700,...,8.678000,12.020000,54.490000,223.600000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,0.000000,11.705000,16.170000,75.100000,420.300000,0.085825,0.062890,0.029320,0.020340,0.161850,...,13.055000,21.045000,84.255000,516.450000,0.114400,0.144950,0.116550,0.063930,0.249500,0.071050
50%,0.000000,13.300000,18.680000,85.980000,551.700000,0.094620,0.090970,0.061540,0.033410,0.179200,...,14.970000,25.220000,97.670000,686.600000,0.130900,0.210100,0.226400,0.098610,0.282700,0.080060
75%,1.000000,15.740000,21.585000,103.750000,767.600000,0.104550,0.131300,0.132350,0.073895,0.195800,...,18.410000,29.690000,124.650000,1031.500000,0.146050,0.341600,0.387200,0.161100,0.317750,0.092070
max,1.000000,28.110000,39.280000,188.500000,2501.000000,0.163400,0.311400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.218400,0.937900,1.252000,0.291000,0.663800,0.173000


In [17]:
#Removing outliers using IQR
Q1 = train_df.quantile(0.25,numeric_only=True)
Q3 = train_df.quantile(0.75,numeric_only=True)
IQR = Q3 - Q1 # difference between the third and first quartiles
train_df = train_df[~((train_df < (Q1 - 1.5 * IQR)) | (train_df > (Q3 + 1.5 * IQR))).any(axis=1)]

In [18]:
#Splititng data into x and y
X = log_reg_data.iloc[:,1:]
y = log_reg_data.iloc[:,0]
#let's split the data into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#Normalizing the data
X_train_normalized = Functions().normalize_features(X_train)
X_test_normalized = Functions().normalize_features(X_test)
# fitting the model
log_reg_model = LogisticRegression(X_train_normalized,y_train)
log_reg_model_params,train_loss,test_loss = log_reg_model.fit()

In [19]:
log_reg_model_preds = log_reg_model.predict(X_test_normalized)


In [45]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.special import expit

# Helper Functions
class Functions:
    ## function for random sampling of data
    def random_sampler(self, X, y, perc=0.1):
        prop = int(len(X) * perc) #proportion of the data to be used for each iteration, here we have set it to 10% of the data
        ind = np.random.choice(range(len(X)), prop, replace=False)
        test_ind = list(set(range(len(X))) - set(ind)) #for using the rest sample as test set
        return X[ind], y[ind], X[test_ind], y[test_ind]
    
    ## function for normalizing the features
    def normalize_features(self, X, append=True):
        X = (X - np.mean(X, 0)) / np.std(X, 0) #normalize the features
        if append:
            X = np.append(np.ones(X.shape[0]).reshape(-1,1), X, 1) #append column of ones for intercept
        return X

class Logistic_Regression():
    def __init__(self, X, y):
        self.X = X
        self.y = y.values.reshape(-1,1)
        
    def fit(self, max_iter=100, tolerance=1e-6, lr=0.001):         
        # Initialize Beta with small random values
        np.random.seed(0)
        B = np.random.randn(self.X.shape[1], 1) * 0.01
        
        # function for updating parameters using newton's method
        x, y, _, _ = Functions().random_sampler(self.X, self.y, perc=0.80)  # use 80-20 in the training too
        
        for it in range(max_iter):
            # calculate training loss
            _, gradient, hessian = self.cross_entropy_loss(x, y, B)
            
            # newton method
            e = -np.linalg.solve(hessian, gradient)
            B = B + lr * e
            
            # stop loop once updation becomes insignificant
            if np.all(np.abs(e) < tolerance):
                break
                
        self.B = B
        return B
    
    def predict(self, X):
        scores = np.matmul(X, self.B)
        # Avoiding division by zero
        exp_scores = np.exp(-scores)
        return np.where(exp_scores < 1e-10, 1.0, 1. / (1. + exp_scores))

    def cross_entropy_loss(self, X, y, B):
        N = len(X)
        p = 1. / (1. + np.exp(-np.dot(X, B)))
        W = np.diag((p * (1 - p)).reshape(-1))
        loss = (-1 / N * (np.dot(y.T, np.log(p)) + np.dot(np.transpose(1 - y), np.log(1 - p))))[0][0]
        gradient = -np.dot(X.T, (y - p))
        hessian = X.T @ W @ X
        return loss, gradient, hessian

## reading the data
log_reg_data = pd.read_csv('logistic.csv')
# Let's replace the categorical variable with M being 1 and B being 0
log_reg_data['Y'] = log_reg_data['Y'].replace({'M':1,'B':0})

# Removing outliers using IQR
Q1 = log_reg_data.quantile(0.25, numeric_only=True)
Q3 = log_reg_data.quantile(0.75, numeric_only=True)
IQR = Q3 - Q1 # difference between the third and first quartiles
log_reg_data = log_reg_data[~((log_reg_data < (Q1 - 1.5 * IQR)) | (log_reg_data > (Q3 + 1.5 * IQR))).any(axis=1)]

# Splititng data into x and y
X = log_reg_data.iloc[:, 1:]
y = log_reg_data.iloc[:, 0]

# Let's split the data into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizing the data
X_train_normalized = Functions().normalize_features(X_train)
X_test_normalized = Functions().normalize_features(X_test)

# Fitting the model
log_reg_model1 = Logistic_Regression(X_train_normalized, y_train)
log_reg_model_params1 = log_reg_model1.fit()



In [48]:
# Calculate training loss for log_reg_model1
train_loss_list1 = []
for x, y in zip(X_train_normalized, y_train):
    y = np.array([y]).reshape(1, 1)  # Convert y to numpy array and reshape
    p = log_reg_model1.predict(x.reshape(1, -1))  # Calculate probability using predict method
    curr_loss, _, _ = log_reg_model1.cross_entropy_loss(x.reshape(1, -1), y, log_reg_model_params1)  # Ensure x is an array
    train_loss_list1.append(curr_loss)

# Calculate test loss for log_reg_model1
test_loss_list1 = []
for x, y in zip(X_test_normalized, y_test):
    y = np.array([y]).reshape(1, 1)  # Convert y to numpy array and reshape
    p = log_reg_model1.predict(x.reshape(1, -1))  # Calculate probability using predict method
    curr_loss, _, _ = log_reg_model1.cross_entropy_loss(x.reshape(1, -1), y, log_reg_model_params1)  # Ensure x is an array
    test_loss_list1.append(curr_loss)


In [49]:
train_loss_list1

[0.4899550861106548,
 0.6468907863195028,
 0.5986691135564758,
 0.6815639765758792,
 0.5569019377024386,
 0.5742134596576609,
 0.6119750555269736,
 0.6771432304378617,
 0.5818859212246589,
 0.6286559743914196,
 0.5883014230944422,
 0.5585517634305328,
 0.5984429047818006,
 0.5801195168534707,
 0.5397610397494413,
 0.567105582704365,
 0.5212148605340438,
 0.5243548852163936,
 0.651996079714507,
 0.658784732636071,
 0.5927552502728928,
 0.5113208098476771,
 0.6719046771890299,
 0.5910890106299694,
 0.4828913726802667,
 0.648064045907902,
 0.4669693852369213,
 0.6240618498009951,
 0.6142157411906369,
 0.6359129794113787,
 0.5374896163881792,
 0.635288779546336,
 0.7179497559299064,
 0.6040093045642512,
 0.6317061541509656,
 0.4848296857888362,
 0.5886360352215703,
 0.5231943577706903,
 0.6434411022723633,
 0.6039800026592133,
 0.6699359755207712,
 0.5647347523615027,
 0.6088093361896832,
 0.6151764257329493,
 0.588874963918671,
 0.6640287812160139,
 0.6321520566791573,
 0.6582870846901662

In [51]:

# # Ensure y_train is extracted as a numpy array
# y_train = y_train.values

# # Calculate training loss
# train_loss_list = []
# for x, y in zip(X_train_normalized, y_train):
#     y = np.array([y])  # Convert y to numpy array
#     p = log_reg_model.predict(x.reshape(1, -1))  # Calculate probability using predict method
#     curr_loss, _, _ = log_reg_model.cross_entropy_loss(x, y, log_reg_model_params)
#     train_loss_list.append(curr_loss)

# # Calculate test loss
# test_loss_list = []
# for x, y in zip(X_test_normalized, y_test):
#     y = np.array([y])  # Convert y to numpy array
#     p = log_reg_model.predict(x.reshape(1, -1))  # Calculate probability using predict method
#     curr_loss, _, _ = log_reg_model.cross_entropy_loss(x, y, log_reg_model_params)
#     test_loss_list.append(curr_loss)

# # Optionally, you can plot or print these losses if needed
# print("Train Loss:", train_loss_list)
# print("Test Loss:", test_loss_list)
